# Ordinary Least Squares (OLS) vs. Gradient Descent (GD)

## 1) Framing the Problem

Multiple regression is a statistical technique used to model the relationship between one dependent variable (or label) and two or more independent variables (or features). It is a powerful tool for understanding how changes in independent variables influence the dependent variable. While this modeling approach can be implemented using various strategies, I will focus on two main methods: 
- **Ordinary Least Squares (OLS)**, which derives the solution through pure mathematical analysis, 
- **Gradient Descent (GD)**, an iterative optimization algorithm often associated with machine learning.

Although Python packages exist for conducting multiple regression, this article will show how to derive the regression parameters using both OLS and GD from scratch. Then, it compare their performance and applicability based on the following criteria:
- **Computational Speed**: How quickly each method converges to a solution, particularly for datasets of varying sizes and dimensionalities.
- **Memory Usage**: The amount of memory required by each approach, especially for large datasets where matrix operations (OLS) may become prohibitive.
- **Convergence Behavior**: The robustness of GD to hyperparameter tuning (e.g., learning rate) and its ability to converge to an optimal solution in different conditions.
- **Numerical Stability**: How each method performs when faced with ill-conditioned data or high multicollinearity among independent variables.
- **Scalability**: Suitability for big data scenarios, where the efficiency of processing may differ significantly between the two methods.
- **Ease of Implementation**: Practical considerations, including code complexity, ease of debugging, and availability of libraries or tools.
- **Accuracy**: The ability of each method to minimize the error term (residuals) and produce reliable parameter estimates under different scenarios.

Finally, the article will explore scenarios where one method is preferable over the other. For example:
- OLS may excel in smaller datasets where computational resources are not constrained.
- GD is often the method of choice for very large datasets or when working with models that extend beyond linear regression, such as neural networks.

By analyzing these aspects, this article aims to provide a comprehensive comparison to help readers choose the appropriate method for their specific use case.

First, let's focus on how the parameters are derived with OLS and GD.

### **How to Use Ordinary Least Squares (OLS)**

#### **0) How to Get Started**

We start with the following equation:

$$Xβ = y$$

Where:

*X* := The design matrix of independent variables\
*β* := The vector of parameters (coefficients to be estimated)\
*y* := The vector of the dependent variable

This is not possible, however, since there is always an error value, known as the *residuals*. 

Thus:

$$X\hat{β} + ε = \hat{y}$$

With that in mind, OLS can be derived through various approaches:

#### **1) Optimization using calculus**  

In this scenario, we minimize the norm of $y - \hat{y}$, which will give us the vector of parameters. $$\min_β |y - \hat{y}| = \min_β |y - X\hat{β}|^2$$

I won't do the math for this, but if you're interested, it requires calculus and is a stronger solution since it follows along the maximum likelihood estimation (MLE).

#### **2) Using the Left Inverse**

Because $Xβ$ is assumed to be full column rank, we can derive *β*.

In other words, if we multiply $Xβ$ by its transpose, we can then multiply it by its inverse.

Thus:

$$X^TXβ = X^Ty$$
$$(X^TX)^{-1}X^TXβ = (X^T X)^{-1}X^Ty$$
$$Iβ = (X^TX)^{-1}X^Ty$$
$$β = (X^T X)^{-1}X^Ty$$

Note:
*I* = The identity matrix

Because β is predicting y, it is better to conclude that this derivation represents $\hat{β}$.

This is not the best approach because we are assuming $Xβ$ is full column rank, which may not be true if there is **Multicollinearity**.

Why? If one parameter is a linear combination of any of the others, then the design matrix $X$ is not of full column rank (e.g. $β_3 = 4*β_1$).

#### **3) Transforming $X$ to Row Canonical Form**

The equation

$$X^TXβ = X^Ty$$

can be row reduced to its canonical form to get the vector of parameters as well.

This will make sense in the next derivation.

#### **4) Using Orthogonal Projections**

$Xβ = y$ can be seen as having a projection and an orthogonal component.

If we view the parameters as predictions, then we have a projection:
$$X\hat{β} = \hat{y}$$

This is also known as $\hat{y}$ projected onto the column space of $X\hat{β}$

The orthogonal component will be the nullspace $X\hat{β}$, which is the residual: $y - \hat{y}$ (or $y - X\hat{β}$)

In other words, the *residual* (orthogonal) and *prediction* (projection) add up to the vector of the dependent variable: y

What now?

Given that $y - X\hat{β}$ is orthogonal to the column space of $X$, then it's orthogonal to each column of $X$:

$$X^T*(y - X\hat{β}) = 0$$
$$X^Ty - X^TX\hat{β} = 0$$
$$X^Ty = X^TX\hat{β}$$
$$(X^TX)^{-1}X^Ty = \hat{β}$$

We could have solved for that using row reduction, where $X^TX$ is augmented by $X^Ty$, then put into row canonical form.

#### **Conclusion**

Regardless of which approach you use with OLS, the coefficients will be:
$$\hat{β} = (X^TX)^{-1}X^Ty$$

<!-- #### **0) How to Get Started**

With GD, you must decide a cost function, AKA a loss or an objective function.

For multiple regression, we start with the mean square error (MSE) as the cost function (it is most common):

$$MSE = \frac{1}{N} \sum_{i=1}^N (y_i - \beta_0 - \mathbf{x}_i^T \boldsymbol{\beta})^2$$

Where:

*N* := Number of data points\
*y_i* := The vector of the dependent variable\
*β_0* := The scalar of the independent variable's intercept\
*β_i* := The vector of parameters (coefficients to be estimated)\
*x_i* := The vector of independent variables

Note: This has not been put in matrix form because GD uses calculus.

At first, you might think, "How can I calculate the residual without first having values for the parameters?"

GD iteratively determines the parameters' values.

You add the value of the partial derivative of the MSE with regard to the particular parameter:

$$\frac{\partial MSE}{\partial \beta_0} = -\frac{2}{N} \sum_{i=1}^N (y_i - \beta_0 - \mathbf{x}_i^T \boldsymbol{\beta_i})$$

$$\frac{\partial MSE}{\partial {\beta_i}} = -\frac{2}{N} \sum_{i=1}^{N}(y_i - \beta_0 + \mathbf{x}_i^T \boldsymbol{\beta_i}) * x_i$$

Now, let's set the parameters to 0 and add a learning rate (α) and number of times (i.e. epochs) to update each parameter.

$\alpha = 0.01$ will be the learning rate, and $t = 100$ will be the number of epochs.

We want to update the model parameters using the following equations:

$$\beta_0^{t+1} = \beta_0 + \alpha \cdot \frac{\partial MSE}{\partial \beta_0}$$

$$\beta_i^{t+1} = \beta_i + \alpha \cdot \frac{\partial MSE}{\partial \beta_i}$$

Substituting the derivatives we found earlier:

$$\beta_0^{t+1} = \beta_0^{t+1} + 0.01 \cdot (-\frac{2}{N} \sum_{i=1}^{N}(y_i - (\beta_0^{t+1} + \beta_i x_i)))$$

$$\beta_i^{t+1} = \beta_i + 0.01 \cdot (-\frac{2}{N} \sum_{i=1}^{N}(y_i - (\boldsymbol{\beta_0} + \beta_i \boldsymbol{x_i})) \cdot \boldsymbol{x_i})$$

Simplifying the equations:

$$\beta_0^{t+1} = \beta_0 - 0.02 \sum_{i=1}^{N}(y_i - (\boldsymbol{\beta_0} + \beta_i \boldsymbol{x_i}))$$

$$\beta_i^{t+1} = \beta_i - 0.02 \sum_{i=1}^{N}(y_i - (\boldsymbol{\beta_0} + \beta_i \boldsymbol{x_i})) \cdot \boldsymbol{x_i}$$ -->

### **How to Use Gradient Descent (GD)**

#### **0) How to Get Started**

To perform Gradient Descent (GD), you must first decide on a cost function, also known as a loss or objective function.

For multiple regression, we typically use the mean squared error (MSE) as the cost function:

$$MSE = \frac{1}{N} \| \mathbf{y} - \beta_0 \mathbf{1} - \mathbf{X} \boldsymbol{\beta} \|^2$$

Where:

*N* := Number (scalar) of data points\
*y* := Vector of dependent variables (N * 1)\
*β_0* := Scalar intercept term\
*1* := Vector of 1's (n * 1)\
*X* := Matrix of independent variables (M * N)\
*β* := Vector of parameters (coefficients to be estimated, N * 1)

At first, you might think, "How can I calculate the residual without first having values for the parameters?"

GD iteratively determines the parameters' values updating the parameters at each step based on the gradient of the cost function, which, in this case, is the MSE.

You add the value of the partial derivative of the MSE with regard to the particular parameter:

$$\frac{\partial MSE}{\partial \beta_0} = -\frac{2}{N} \mathbf{1}^T (\mathbf{y} - \beta_0 \mathbf{1} - \mathbf{X} \boldsymbol{\beta})$$

$$\frac{\partial MSE}{\partial \boldsymbol{\beta}} = -\frac{2}{N} \mathbf{X}^T (\mathbf{y} - \beta_0 \mathbf{1} - \mathbf{X} \boldsymbol{\beta})$$

Now, let's set the parameters to 0 and add a learning rate (α) and number of times (i.e. epochs) to update each parameter.

$α = 0.01$ will be the learning rate, and $t = 100$ will be the number of epochs.

The updates for the parameters are:

$$\beta_0^{(t+1)} = \beta_0^{(t)} - \alpha \cdot \frac{\partial MSE}{\partial \beta_0}$$

$$\boldsymbol{\beta}^{(t+1)} = \boldsymbol{\beta}^{(t)} - \alpha \cdot \frac{\partial MSE}{\partial \boldsymbol{\beta}}$$

Substituting the derivatives into the update equations:

$$\beta_0^{(t+1)} = \beta_0^{(t)} + \frac{2 \alpha}{N} \mathbf{1}^T (\mathbf{y} - \beta_0^{(t)} \mathbf{1} - \mathbf{X} \boldsymbol{\beta}^{(t)})$$

$$\boldsymbol{\beta}^{(t+1)} = \boldsymbol{\beta}^{(t)} + \frac{2 \alpha}{N} \mathbf{X}^T (\mathbf{y} - \beta_0^{(t)} \mathbf{1} - \mathbf{X} \boldsymbol{\beta}^{(t)})$$

#### **Conclusion**

The expressions $\beta_0^{(t+1)}$ and $\boldsymbol{\beta}^{(t+1)}$ would give us the intercept and vector of parameters.

## 2) Gathering the Data

The dataset comes from [Kaggle](https://www.kaggle.com/datasets/nikhil7280/student-performance-multiple-linear-regression).

The dataset examines the factors influencing academic student performance, including:
- Hours Studied: The total number of hours spent studying by each student.
- Previous Scores: The scores obtained by students in previous tests.
- Extracurricular Activities: Whether the student participates in extracurricular activities (Yes or No).
- Sleep Hours: The average number of hours of sleep the student had per day.
- Sample Question Papers Practiced: The number of sample question papers the student practiced.

The dataset consists of 10,000 student records, with each record containing information about various predictors and a performance index.

The target variable, performance index, measures each student's overall performance. The index, rounded to the nearest integer ranges from 10 to 100, with higher values indicating better performance.

In [1]:
# The dataset will come from Kaggle:

import kagglehub

# Download latest version
path = kagglehub.dataset_download("nikhil7280/student-performance-multiple-linear-regression")
print("Path to dataset files:", path)

100%|██████████| 48.5k/48.5k [00:00<00:00, 21.3MB/s]

Extracting files...
Path to dataset files: /Users/alexdubro/.cache/kagglehub/datasets/nikhil7280/student-performance-multiple-linear-regression/versions/1


In [3]:
import matplotlib.pyplot as plt
import numpy as np 
import os
import pandas as pd 
import seaborn as sns 

dataset_path = os.path.join(path, 'Student_Performance.csv')

raw_data = pd.read_csv(dataset_path)
raw_data.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0


## 3) Cleaning/preprocessing data

In [ ]:
# 1) Removing null data

raw_data.dropna(inplace=True)

# 2) Removing duplicates

raw_data.drop_duplicates(inplace=True)

Because there is a categorical variable, the data must be dummy coded.

In [18]:
raw_data['Extracurricular Activities'].value_counts()

Extracurricular Activities
No     4986
Yes    4887
Name: count, dtype: int64

In [19]:
updated_data = raw_data.copy()  # making a copy so that our original data remains intact

In [22]:
updated_data['Extracurricular Activities'] = raw_data['Extracurricular Activities'].map({'No': 0, 'Yes': 1}) # dummy coding the data

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,1,9,1,91.0
1,4,82,0,4,2,65.0
2,8,51,1,7,2,45.0
3,5,52,1,5,2,36.0
4,7,75,0,8,5,66.0


In [41]:
data_0 = updated_data[updated_data['Extracurricular Activities'] == 0]
data_0.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
1,4,82,0,4,2,65.0
4,7,75,0,8,5,66.0
5,3,78,0,9,6,61.0
8,5,77,0,8,2,61.0
9,4,89,0,4,0,69.0


In [ ]:
X_0 = data_0.drop('Performance Index', axis=1) # Extracurricular Activities: 'No': 0
y_0 = data_0['Performance Index']

In [46]:
data_1 = updated_data[updated_data['Extracurricular Activities'] == 1]
data_1.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,1,9,1,91.0
2,8,51,1,7,2,45.0
3,5,52,1,5,2,36.0
6,7,73,1,5,6,63.0
7,8,45,1,4,6,42.0


In [47]:
X_1 = data_1.drop('Performance Index', axis=1)
y_1 = data_1['Performance Index']

## 4) Visualizing the Data

In [54]:
updated_data.describe()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
count,9873.000000,9873.000000,9873.000000,9873.000000,9873.000000,9873.000000
mean,4.992100,69.441102,0.494986,6.531652,4.583004,55.216651
std,2.589081,17.325601,0.500000,1.697683,2.867202,19.208570
min,1.000000,40.000000,0.000000,4.000000,0.000000,10.000000
25%,3.000000,54.000000,0.000000,5.000000,2.000000,40.000000
50%,5.000000,69.000000,0.000000,7.000000,5.000000,55.000000
75%,7.000000,85.000000,1.000000,8.000000,7.000000,70.000000
max,9.000000,99.000000,1.000000,9.000000,9.000000,100.000000


In [ ]:
plt.figure(figsize=(12, 8))

plt.hist(df['MEDV'], bins=50, ec='black', rwidth=0.5)
plt.xlabel('Price in thousands')
plt.ylabel('Number of houses')
plt.title("Counts of Housing Prices")
plt.show()

## 5) Training & Building the Algorithm: Deriving Parameters Using OLS & GD 

## Ordinary Least Squares (OLS)

We will be using the following equation to derive the parameters for OLS:

$$\hat{β} = (X^TX)^{-1}X^Ty$$

In [43]:
# y_0 = data_0['Performance Index']
# X_0 = data_0.drop('Performance Index', axis=1)
# x_0_transpose = np.transpose(X_0)

# beta = np.linalg.inv(X_0.T @ X_0) @ X_0.T @ y_0

# Correct implementation of normal equations
X_0 = data_0.drop('Performance Index', axis=1)
y_0 = data_0['Performance Index']

# Add a column of 1s for the intercept term
X_0_with_intercept = np.column_stack([np.ones(len(X_0)), X_0])

# Use matrix multiplication and matrix inverse
beta = np.linalg.inv(X_0_with_intercept.T @ X_0_with_intercept) @ X_0_with_intercept.T @ y_0

LinAlgError: Singular matrix

Because of Extracurricular Activities (namely, the dummy variable), the matrix is not full column rank. Therefore, other approaches must be taken to calculate its parameters.

## 1) Ridge Regression

In [44]:
def ridge_regression(X, y, lambda_=1.0):
    # Add intercept column
    X_with_intercept = np.column_stack([np.ones(len(X)), X])
    
    # Create identity matrix (skip first element for intercept)
    I = np.eye(X_with_intercept.shape[1])
    I[0,0] = 0  # Don't regularize intercept
    
    # Solve (X^T X + λI)^(-1) X^T y
    beta = np.linalg.inv(X_with_intercept.T @ X_with_intercept + lambda_ * I) @ X_with_intercept.T @ y
    
    return beta

parameters = ridge_regression(X_0, y_0, lambda_=1.0)
print(parameters)

[-34.02208685   2.8404049    1.01806564   0.           0.48233107
   0.19807239]


## 2) Singular Value Decomposition (SVD)

In [45]:
U, s, Vt = np.linalg.svd(X_0_with_intercept, full_matrices=False)

# Create a diagonal matrix with reciprocals of singular values
# Add a small threshold to handle near-zero singular values
s_inv = np.zeros_like(s)
threshold = 1e-10  # You can adjust this threshold
s_inv[s > threshold] = 1 / s[s > threshold]

# Construct the pseudoinverse
# V * Σ⁺ * U^T
pseudoinverse = Vt.T @ np.diag(s_inv) @ U.T

# Compute beta
beta = pseudoinverse @ y_0
print(beta)

[-3.40227939e+01  2.84048888e+00  1.01806649e+00 -4.12559464e-17
  4.82363491e-01  1.98076137e-01]


In [ ]:
# y_0 = data_0['Performance Index']
# X_0 = data_0.drop('Performance Index', axis=1)
# x_0_transpose = np.transpose(X_0)

beta_11 = np.linalg.inv(X_0.T @ X_0) @ X_0.T @ y_0
print(beta_11)

LinAlgError: Singular matrix

In [53]:
# Correct implementation of normal equations
# X_1 = data_1.drop('Performance Index', axis=1)
# y_1 = data_1['Performance Index']

# Add a column of 1s for the intercept term
X_1_with_intercept = np.column_stack([np.ones(len(X_1)), X_1])

# Use matrix multiplication and matrix inverse
beta_22 = np.linalg.inv(X_1_with_intercept.T @ X_1_with_intercept) @ X_1_with_intercept.T @ y_1
print(beta_22)

[ 9.38976005e+17 -4.12495736e+00  1.98060613e+00 -9.38976005e+17
  3.98771005e+01  0.00000000e+00]


In [50]:
def ridge_regression(X, y, lambda_=1.0):
    # Add intercept column
    X_with_intercept = np.column_stack([np.ones(len(X)), X])
    
    # Create identity matrix (skip first element for intercept)
    I = np.eye(X_with_intercept.shape[1])
    I[0,0] = 0  # Don't regularize intercept
    
    # Solve (X^T X + λI)^(-1) X^T y
    beta = np.linalg.inv(X_with_intercept.T @ X_with_intercept + lambda_ * I) @ X_with_intercept.T @ y
    
    return beta

parameters = ridge_regression(X_1, y_1, lambda_=1.0)
print(parameters)

[-3.34984716e+01  2.86538453e+00  1.01852677e+00  1.52518478e-11
  4.78547138e-01  1.89582109e-01]


In [51]:
U, s, Vt = np.linalg.svd(X_1_with_intercept, full_matrices=False)

# Create a diagonal matrix with reciprocals of singular values
# Add a small threshold to handle near-zero singular values
s_inv = np.zeros_like(s)
threshold = 1e-10  # You can adjust this threshold
s_inv[s > threshold] = 1 / s[s > threshold]

# Construct the pseudoinverse
# V * Σ⁺ * U^T
pseudoinverse = Vt.T @ np.diag(s_inv) @ U.T

# Compute beta
beta = pseudoinverse @ y_1
print(beta)

[-16.74960152   2.865473     1.0185275  -16.74960152   0.47858157
   0.18958515]
